# SageMaker Serverless Inference
## XGBoost Regression Example

Amazon SageMaker Serverless Inference is a purpose-built inference option that makes it easy for customers to deploy and scale ML models. Serverless Inference is ideal for workloads which have idle periods between traffic spurts and can tolerate cold starts. Serverless endpoints also automatically launch compute resources and scale them in and out depending on traffic, eliminating the need to choose instance types or manage scaling policies.

For this notebook we'll be working with the SageMaker XGBoost Algorithm to train a model and then deploy a serverless endpoint. We will be using the public S3 Abalone regression dataset for this example.

<b>Notebook Setting</b>
- <b>SageMaker Classic Notebook Instance</b>: ml.m5.xlarge Notebook Instance & conda_python3 Kernel
- <b>SageMaker Studio</b>: Python 3 (Data Science)
- <b>Regions Available</b>: SageMaker Serverless Inference is currently available in the following regions: US East (Northern Virginia), US East (Ohio), US West (Oregon), EU (Ireland), Asia Pacific (Tokyo) and Asia Pacific (Sydney)

## Table of Contents
- Setup
- Model Training
- Deployment
    - Model Creation
    - Endpoint Configuration (Adjust for Serverless)
    - Serverless Endpoint Creation
    - Endpoint Invocation
- Cleanup

## Setup

For testing you need to properly configure your Notebook Role to have <b>SageMaker Full Access</b>.

Let's start by installing preview wheels of the Python SDK, boto and aws cli

In [1]:
# Fallback in case wheels are unavailable
! pip install sagemaker botocore boto3 awscli --upgrade

     |████████████████████████████████| 466 kB 6.5 MB/s eta 0:00:01
     |████████████████████████████████| 8.4 MB 75.4 MB/s eta 0:00:01
     |████████████████████████████████| 131 kB 93.6 MB/s eta 0:00:01
     |████████████████████████████████| 3.8 MB 52.6 MB/s eta 0:00:01
  Created wheel for sagemaker: filename=sagemaker-2.70.0-py2.py3-none-any.whl size=649170 sha256=f7043c40230b66938a46d8d8ebd61b9d70d67e92a5356f8f8196c40f539c6c4f
  Stored in directory: /home/ec2-user/.cache/pip/wheels/da/11/20/c45ef599886a2b1399effa68f80b98b2166dc624e19636c303
Successfully built sagemaker
  Attempting uninstall: botocore
    Found existing installation: botocore 1.22.11
    Uninstalling botocore-1.22.11:
      Successfully uninstalled botocore-1.22.11
  Attempting uninstall: boto3
    Found existing installation: boto3 1.19.3
    Uninstalling boto3-1.19.3:
      Successfully uninstalled boto3-1.19.3
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.66.0
    Uninstalling 

In [2]:
import subprocess


def execute_cmd(cmd):
    print(cmd)
    output = subprocess.getstatusoutput(cmd)
    return output


def _download_from_s3(_file_path):
    _path = f"s3://reinvent21-sm-rc-wheels/{_file_path}"
    print(f"Path is {_path}")
    ls_cmd = f"aws s3 ls {_path}"
    print(execute_cmd(ls_cmd))

    cmd = f"aws s3 cp {_path} /tmp/"
    print("Downloading: ", cmd)
    return execute_cmd(cmd)


def _install_wheel(wheel_name):
    cmd = f"pip install --no-deps --log /tmp/output3.log /tmp/{wheel_name} --force-reinstall"

    ret = execute_cmd(cmd)

    _name = wheel_name.split(".")[0]
    _, _version = execute_cmd(f"python -c 'import {_name}; print({_name}.__version__)'")

    for package in ["botocore", "sagemaker", "boto3", "awscli"]:
        print(execute_cmd(f"python -c 'import {package}; print({package}.__version__)'"))

    print(f"Installed {_name}:{_version}")

    return ret


def install_sm_py_sdk():
    pySDK_name = "sagemaker.tar.gz"

    exit_code, _ = _download_from_s3("dist/sagemaker.tar.gz")

    if not exit_code:
        _install_wheel(pySDK_name)
    else:
        print(f"'{pySDK_name}' is not present in S3 Bucket. Installing from public PyPi...")
        execute_cmd("pip install sagemaker")


def install_boto_wheels():
    WHEELS = ["botocore.tar.gz", "boto3.tar.gz", "awscli.tar.gz"]

    for wheel_name in WHEELS:
        _path = f"boto3/{wheel_name}"
        exit_code, _ = _download_from_s3(_path)

        if not exit_code:
            _install_wheel(wheel_name)
        else:
            print(f"'{wheel_name}' is not present in S3 Bucket. Ignoring...")


install_boto_wheels()
install_sm_py_sdk()

Path is s3://reinvent21-sm-rc-wheels/boto3/botocore.tar.gz
aws s3 ls s3://reinvent21-sm-rc-wheels/boto3/botocore.tar.gz
(255, '\nAn error occurred (AccessDenied) when calling the ListObjectsV2 operation: Access Denied')
Downloading:  aws s3 cp s3://reinvent21-sm-rc-wheels/boto3/botocore.tar.gz /tmp/
aws s3 cp s3://reinvent21-sm-rc-wheels/boto3/botocore.tar.gz /tmp/
pip install --no-deps --log /tmp/output3.log /tmp/botocore.tar.gz --force-reinstall
python -c 'import botocore; print(botocore.__version__)'
python -c 'import botocore; print(botocore.__version__)'
(0, '1.23.7')
python -c 'import sagemaker; print(sagemaker.__version__)'
(0, '2.70.0')
python -c 'import boto3; print(boto3.__version__)'
(0, '1.20.18')
python -c 'import awscli; print(awscli.__version__)'
(0, '1.22.19')
Installed botocore:1.23.7
Path is s3://reinvent21-sm-rc-wheels/boto3/boto3.tar.gz
aws s3 ls s3://reinvent21-sm-rc-wheels/boto3/boto3.tar.gz
(255, '\nAn error occurred (AccessDenied) when calling the ListObjectsV2 

In [3]:
# Setup clients
import boto3

client = boto3.client(service_name="sagemaker")
runtime = boto3.client(service_name="sagemaker-runtime")

### SageMaker Setup
To begin, we import the AWS SDK for Python (Boto3) and set up our environment, including an IAM role and an S3 bucket to store our data.

In [4]:
import boto3
import sagemaker
from sagemaker.estimator import Estimator

boto_session = boto3.session.Session()
region = boto_session.region_name
print(region)

sagemaker_session = sagemaker.Session()
base_job_prefix = "xgboost-example"
role = sagemaker.get_execution_role()
print(role)

default_bucket = sagemaker_session.default_bucket()
s3_prefix = base_job_prefix

training_instance_type = "ml.m5.xlarge"

us-west-2
arn:aws:iam::474422712127:role/sagemaker-role-BYOC


Retrieve the Abalone dataset from a publicly hosted S3 bucket.

In [5]:
# retrieve data
!aws s3 cp s3://sagemaker-sample-files/datasets/tabular/uci_abalone/train_csv/abalone_dataset1_train.csv .

download: s3://sagemaker-sample-files/datasets/tabular/uci_abalone/train_csv/abalone_dataset1_train.csv to ./abalone_dataset1_train.csv


Upload the Abalone dataset to the default S3 bucket.

In [6]:
# upload data to S3
!aws s3 cp abalone_dataset1_train.csv s3://{default_bucket}/xgboost-regression/train.csv

upload: ./abalone_dataset1_train.csv to s3://sagemaker-us-west-2-474422712127/xgboost-regression/train.csv


## Model Training

Now, we train an ML model using the XGBoost Algorithm. In this example, we use a SageMaker-provided [XGBoost](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html) container image and configure an estimator to train our model.

In [7]:
from sagemaker.inputs import TrainingInput

training_path = f"s3://{default_bucket}/xgboost-regression/train.csv"
train_input = TrainingInput(training_path, content_type="text/csv")

In [8]:
model_path = f"s3://{default_bucket}/{s3_prefix}/xgb_model"

# retrieve xgboost image
image_uri = sagemaker.image_uris.retrieve(
    framework="xgboost",
    region=region,
    version="1.0-1",
    py_version="py3",
    instance_type=training_instance_type,
)

# Configure Training Estimator
xgb_train = Estimator(
    image_uri=image_uri,
    instance_type=training_instance_type,
    instance_count=1,
    output_path=model_path,
    sagemaker_session=sagemaker_session,
    role=role,
)

# Set Hyperparameters
xgb_train.set_hyperparameters(
    objective="reg:linear",
    num_round=50,
    max_depth=5,
    eta=0.2,
    gamma=4,
    min_child_weight=6,
    subsample=0.7,
    silent=0,
)

Train the model on the Abalone dataset.

In [9]:
# Fit model
xgb_train.fit({"train": train_input})

2021-12-02 20:12:31 Starting - Starting the training job...
2021-12-02 20:12:56 Starting - Launching requested ML instancesProfilerReport-1638475951: InProgress
......
2021-12-02 20:13:56 Starting - Preparing the instances for training......
2021-12-02 20:15:00 Downloading - Downloading input data...
2021-12-02 20:15:22 Training - Downloading the training image..INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value reg:linear to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
[20:15:38] 2923x8 matrix with 23384 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
INFO:root:Single node training.
INFO:root:Train matri

## Deployment

After training the model, retrieve the model artifacts so that we can deploy the model to an endpoint.

In [10]:
# Retrieve model data from training job
model_artifacts = xgb_train.model_data
model_artifacts

's3://sagemaker-us-west-2-474422712127/xgboost-example/xgb_model/sagemaker-xgboost-2021-12-02-20-12-31-191/output/model.tar.gz'

### Model Creation
Create a model by providing your model artifacts, the container image URI, environment variables for the container (if applicable), a model name, and the SageMaker IAM role.

In [11]:
from time import gmtime, strftime

model_name = "xgboost-serverless" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("Model name: " + model_name)

# dummy environment variables
byo_container_env_vars = {"SAGEMAKER_CONTAINER_LOG_LEVEL": "20", "SOME_ENV_VAR": "myEnvVar"}

create_model_response = client.create_model(
    ModelName=model_name,
    Containers=[
        {
            "Image": image_uri,
            "Mode": "SingleModel",
            "ModelDataUrl": model_artifacts,
            "Environment": byo_container_env_vars,
        }
    ],
    ExecutionRoleArn=role,
)

print("Model Arn: " + create_model_response["ModelArn"])

Model name: xgboost-serverless2021-12-02-20-16-42
Model Arn: arn:aws:sagemaker:us-west-2:474422712127:model/xgboost-serverless2021-12-02-20-16-42


### Endpoint Configuration Creation

This is where you can adjust the <b>Serverless Configuration</b> for your endpoint. The current max concurrent invocations for a single endpoint, known as <b>MaxConcurrency</b>, can be any value from <b>1 to 50</b>, and <b>MemorySize</b> can be any of the following: <b>1024 MB, 2048 MB, 3072 MB, 4096 MB, 5120 MB, or 6144 MB</b>.

In [12]:
xgboost_epc_name = "xgboost-serverless-epc" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

endpoint_config_response = client.create_endpoint_config(
    EndpointConfigName=xgboost_epc_name,
    ProductionVariants=[
        {
            "VariantName": "byoVariant",
            "ModelName": model_name,
            "ServerlessConfig": {
                "MemorySizeInMB": 4096,
                "MaxConcurrency": 1,
            },
        },
    ],
)

print("Endpoint Configuration Arn: " + endpoint_config_response["EndpointConfigArn"])

Endpoint Configuration Arn: arn:aws:sagemaker:us-west-2:474422712127:endpoint-config/xgboost-serverless-epc2021-12-02-20-16-46


### Serverless Endpoint Creation
Now that we have an endpoint configuration, we can create a serverless endpoint and deploy our model to it. When creating the endpoint, provide the name of your endpoint configuration and a name for the new endpoint.

In [13]:
endpoint_name = "xgboost-serverless-ep" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

create_endpoint_response = client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=xgboost_epc_name,
)

print("Endpoint Arn: " + create_endpoint_response["EndpointArn"])

Endpoint Arn: arn:aws:sagemaker:us-west-2:474422712127:endpoint/xgboost-serverless-ep2021-12-02-20-16-50


Wait until the endpoint status is InService before invoking the endpoint.

In [14]:
# wait for endpoint to reach a terminal state (InService) using describe endpoint
import time

describe_endpoint_response = client.describe_endpoint(EndpointName=endpoint_name)

while describe_endpoint_response["EndpointStatus"] == "Creating":
    describe_endpoint_response = client.describe_endpoint(EndpointName=endpoint_name)
    print(describe_endpoint_response["EndpointStatus"])
    time.sleep(15)

describe_endpoint_response

Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
InService


{'EndpointName': 'xgboost-serverless-ep2021-12-02-20-16-50',
 'EndpointArn': 'arn:aws:sagemaker:us-west-2:474422712127:endpoint/xgboost-serverless-ep2021-12-02-20-16-50',
 'EndpointConfigName': 'xgboost-serverless-epc2021-12-02-20-16-46',
 'ProductionVariants': [{'VariantName': 'byoVariant',
   'DeployedImages': [{'SpecifiedImage': '246618743249.dkr.ecr.us-west-2.amazonaws.com/sagemaker-xgboost:1.0-1-cpu-py3',
     'ResolvedImage': '246618743249.dkr.ecr.us-west-2.amazonaws.com/sagemaker-xgboost@sha256:04889b02181f14632e19ef6c2a7d74bfe699ff4c7f44669a78834bc90b77fe5a',
     'ResolutionTime': datetime.datetime(2021, 12, 2, 20, 16, 55, 767000, tzinfo=tzlocal())}],
   'CurrentWeight': 1.0,
   'DesiredWeight': 1.0,
   'CurrentInstanceCount': 0,
   'CurrentServerlessConfig': {'MemorySizeInMB': 4096, 'MaxConcurrency': 1}}],
 'EndpointStatus': 'InService',
 'CreationTime': datetime.datetime(2021, 12, 2, 20, 16, 51, 47000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2021, 12, 2, 20

### Endpoint Invocation
Invoke the endpoint by sending a request to it. The following is a sample data point grabbed from the CSV file downloaded from the public Abalone dataset.

In [15]:
response = runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=b".345,0.224414,.131102,0.042329,.279923,-0.110329,-0.099358,0.0",
    ContentType="text/csv",
)

print(response["Body"].read())

b'4.566554546356201'


## Clean Up
Delete any resources you created in this notebook that you no longer wish to use.

In [16]:
client.delete_model(ModelName=model_name)
client.delete_endpoint_config(EndpointConfigName=xgboost_epc_name)
client.delete_endpoint(EndpointName=endpoint_name)

{'ResponseMetadata': {'RequestId': '069228c0-9462-4389-85fb-2c90377ec818',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '069228c0-9462-4389-85fb-2c90377ec818',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Thu, 02 Dec 2021 20:20:21 GMT'},
  'RetryAttempts': 0}}